In [3]:
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import itertools
import pandas as pd
from more_itertools import locate
from torch_scatter import scatter
import torch
from torch_geometric.data import Data
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import to_networkx
from torch_geometric.transforms import RandomNodeSplit

In [4]:
# Import IHS data
class IHSData(object):
    def __init__(self):
        product_file = "C:/Users\lukec\OneDrive - University of Cambridge\PhD\Data\IHS/US\products.csv"
        material_file = "C:/Users\lukec\OneDrive - University of Cambridge\PhD\Data\IHS/US\materials.csv"
        self.products = pd.read_csv(product_file,index_col=0).reset_index(drop=True)
        self.materials = pd.read_csv(material_file,index_col=0).reset_index(drop=True)
        # self.edgeColumns = ['Process','Geography']
        # self.weights = [10,1]
        # self.adjMatrix, self.edgeWeights = pd_to_adj_matrix(self.products,self.edgeColumns,self.weights)
        # self.nodeTypes = self.products['Investment (MM US$)'].values
        # self.valueColumns = ['Variable Cost','Fixed Costs','Overhead + Tax, Ins.','Depreciation','G&A, Sales, Res.']
        #self.products.iloc[:,np.where(['float' in i or 'int' in i for i in self.products.dtypes.astype(str)])[0]].values

    # def to_pyg_graph(self):
    #     edge_index = torch.tensor(self.adjMatrix, dtype=torch.long)
    #     x = torch.tensor(self.products[self.valueColumns].values, dtype=torch.float)
    #     y = torch.tensor(np.digitize(self.nodeTypes,bins=np.linspace(min(self.nodeTypes),max(self.nodeTypes),4))-1, dtype=torch.long)
    #     return Data(x=x, edge_index=edge_index, y=y)

In [28]:
class EmissionData(object):
    def __init__(self):
        folderPath = "C:/Users\lukec\OneDrive - University of Cambridge\PhD\Data\EcoInvent\Basic_chemicals_201\GWP100a_IPCC2013.csv"
        processes = pd.read_csv(folderPath,index_col=0)
        self.emissions = processes[~ processes['generalComment'].str.contains('market')].sort_values('name')

        merge = pd.merge(ihs.products,ipcc.emissions, left_on=ihs.products['Name'].str.lower(), right_on=ipcc.emissions['name'].str.lower(), how="left").dropna()
        merge.drop_duplicates('Code')

In [29]:
ihs = IHSData()
ipcc = EmissionData()

In [30]:
ihs.products

,Code,Geography,Data Version,Research Year,Process,Unit,Base Capacity (MM unit/year),Investment (MM US$),Type,Name,...,Fixed Costs,"Overhead + Tax, Ins.",Plant Cash Cost,Depreciation,Plant Gate Costs,"G&A, Sales, Res.",Production Cost,ROI (15%),Product Value,Product Price
0,1M-981,"United States, Gulf Coast",2021 Q3,2002.0,ETHYLENE BY THE UOP/HYDRO METHANOL TO OLEFINS ...,KG,400.068470,884.905431,Product,ETHYLENE,...,5.893464,6.625514,34.643831,22.118850,56.762680,2.781679,59.544360,33.178274,92.722634,100.420560
1,1M-1595,"United States, Gulf Coast",2021 Q3,2014.0,ETHYLENE FROM 100% ETHANE,KG,1499.998216,2603.121527,Product,ETHYLENE,...,5.280392,5.692002,32.993959,17.354164,50.348123,2.362249,52.710372,26.031246,78.741618,100.420560
2,1M-1596,"United States, Gulf Coast",2021 Q3,2014.0,ETHYLENE FROM 50:50 ETHANE:PROPANE MIX,KG,1499.998216,2889.486683,Product,ETHYLENE,...,5.825852,6.292007,23.815590,19.263267,43.078858,2.225993,45.304850,28.894901,74.199751,100.420560
3,1M-242,"United States, Gulf Coast",2021 Q3,1991.0,ETHYLENE FROM ATMOSPHERIC GAS OIL BY MILLISECO...,KG,498.951607,1703.670857,Product,ETHYLENE,...,10.504545,10.747590,62.747181,34.145012,96.892193,4.580713,101.472905,51.217518,152.690423,100.420560
4,1M-1824,"United States, Gulf Coast",2021 Q3,2017.0,ETHYLENE FROM CONDENSATE WITH FRONT-END DEPROP...,KG,1000.171176,2816.453891,Product,ETHYLENE,...,8.375398,8.590006,-14.352858,28.159719,13.806860,2.949813,16.756673,42.239578,58.996251,100.420560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,1M-1285,"United States, Gulf Coast",2021 Q3,2006.0,BIMODAL HDPE PRODUCTION BY BASELL’S SPHERILENE...,KG,350.173310,385.298067,Product,"POLYETHYLENE, HDPE BM",...,2.934592,3.500520,114.751364,11.003068,125.754431,7.487318,133.241749,16.504602,149.746351,187.591339
278,1M-1598,"United States, Gulf Coast",2021 Q3,2014.0,"BIO-BASED 1,3-BUTADIENE BY COBALT CATALYST",KG,99.790321,866.749539,Product,BUTADIENE,...,35.317036,33.136856,791.590453,86.857075,878.447527,53.091218,931.538745,130.285612,1061.824357,189.222760
279,1M-1600,"United States, Gulf Coast",2021 Q3,2014.0,"BIO-BASED 1,3-BUTADIENE PRODUCTION BY A DIRECT...",KG,100.017118,341.238624,Product,BUTADIENE,...,13.935629,13.624039,476.866543,34.118022,510.984565,29.587453,540.572018,51.177033,591.749051,189.222760
280,1M-1599,"United States, Gulf Coast",2021 Q3,2014.0,"BIO-BASED 1,3-BUTADIENE PRODUCTION BY AN INDIR...",KG,100.017118,336.193597,Product,BUTADIENE,...,12.439527,12.924715,527.211792,33.613606,560.825398,32.170832,592.996230,50.420409,643.416639,189.222760


In [31]:
merge = pd.merge(ihs.products,ipcc.emissions, left_on=ihs.products['Name'].str.lower(), right_on=ipcc.emissions['name'].str.lower(), how="left").dropna()

In [35]:
merge.dropna()

,key_0,Code,Geography,Data Version,Research Year,Process,Unit,Base Capacity (MM unit/year),Investment (MM US$),Type,...,Plant Gate Costs,"G&A, Sales, Res.",Production Cost,ROI (15%),Product Value,Product Price,name,generalComment,location,CO2e
60,butadiene,1M-1437,"United States, Gulf Coast",2021 Q3,2012.0,"1,3-BUTADIENE BY BUTANE DEHYDROGENATION PROCESS",KG,99.790321,434.326134,Product,...,206.300256,8.399569,214.699825,65.285811,279.985635,189.222760,butadiene,Data are derived from the Eco-profiles of the ...,RER,1.200719
61,butadiene,1M-1437,"United States, Gulf Coast",2021 Q3,2012.0,"1,3-BUTADIENE BY BUTANE DEHYDROGENATION PROCESS",KG,99.790321,434.326134,Product,...,206.300256,8.399569,214.699825,65.285811,279.985635,189.222760,butadiene,Data are derived from the Eco-profiles of the ...,RoW,1.201352
62,butadiene,1M-1438,"United States, Gulf Coast",2021 Q3,2012.0,"1,3-BUTADIENE BY OXIDATIVE DEHYDROGENATION",KG,99.790321,227.181648,Product,...,156.621205,5.900105,162.521310,34.148850,196.670160,189.222760,butadiene,Data are derived from the Eco-profiles of the ...,RER,1.200719
63,butadiene,1M-1438,"United States, Gulf Coast",2021 Q3,2012.0,"1,3-BUTADIENE BY OXIDATIVE DEHYDROGENATION",KG,99.790321,227.181648,Product,...,156.621205,5.900105,162.521310,34.148850,196.670160,189.222760,butadiene,Data are derived from the Eco-profiles of the ...,RoW,1.201352
83,"2,6-di-tert-butylphenol",1M-168,"United States, Gulf Coast",2021 Q3,1993.0,"2,6-DI-TERT-BUTYLPHENOL FROM PHENOL AND ISOBUT...",KG,13.607771,37.618692,Product,...,248.347884,51.143893,299.491776,41.467509,340.959286,255.736224,"2,6-di-tert-butylphenol","2,6-Di-tert-Butylphenol is an essential input ...",GLO,3.513587
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324,butadiene,1M-1598,"United States, Gulf Coast",2021 Q3,2014.0,"BIO-BASED 1,3-BUTADIENE BY COBALT CATALYST",KG,99.790321,866.749539,Product,...,878.447527,53.091218,931.538745,130.285612,1061.824357,189.222760,butadiene,Data are derived from the Eco-profiles of the ...,RoW,1.201352
325,butadiene,1M-1600,"United States, Gulf Coast",2021 Q3,2014.0,"BIO-BASED 1,3-BUTADIENE PRODUCTION BY A DIRECT...",KG,100.017118,341.238624,Product,...,510.984565,29.587453,540.572018,51.177033,591.749051,189.222760,butadiene,Data are derived from the Eco-profiles of the ...,RER,1.200719
326,butadiene,1M-1600,"United States, Gulf Coast",2021 Q3,2014.0,"BIO-BASED 1,3-BUTADIENE PRODUCTION BY A DIRECT...",KG,100.017118,341.238624,Product,...,510.984565,29.587453,540.572018,51.177033,591.749051,189.222760,butadiene,Data are derived from the Eco-profiles of the ...,RoW,1.201352
327,butadiene,1M-1599,"United States, Gulf Coast",2021 Q3,2014.0,"BIO-BASED 1,3-BUTADIENE PRODUCTION BY AN INDIR...",KG,100.017118,336.193597,Product,...,560.825398,32.170832,592.996230,50.420409,643.416639,189.222760,butadiene,Data are derived from the Eco-profiles of the ...,RER,1.200719


In [33]:
merge.drop_duplicates('Code')

,key_0,Code,Geography,Data Version,Research Year,Process,Unit,Base Capacity (MM unit/year),Investment (MM US$),Type,...,Plant Gate Costs,"G&A, Sales, Res.",Production Cost,ROI (15%),Product Value,Product Price,name,generalComment,location,CO2e
60,butadiene,1M-1437,"United States, Gulf Coast",2021 Q3,2012.0,"1,3-BUTADIENE BY BUTANE DEHYDROGENATION PROCESS",KG,99.790321,434.326134,Product,...,206.300256,8.399569,214.699825,65.285811,279.985635,189.222760,butadiene,Data are derived from the Eco-profiles of the ...,RER,1.200719
62,butadiene,1M-1438,"United States, Gulf Coast",2021 Q3,2012.0,"1,3-BUTADIENE BY OXIDATIVE DEHYDROGENATION",KG,99.790321,227.181648,Product,...,156.621205,5.900105,162.521310,34.148850,196.670160,189.222760,butadiene,Data are derived from the Eco-profiles of the ...,RER,1.200719
83,"2,6-di-tert-butylphenol",1M-168,"United States, Gulf Coast",2021 Q3,1993.0,"2,6-DI-TERT-BUTYLPHENOL FROM PHENOL AND ISOBUT...",KG,13.607771,37.618692,Product,...,248.347884,51.143893,299.491776,41.467509,340.959286,255.736224,"2,6-di-tert-butylphenol","2,6-Di-tert-Butylphenol is an essential input ...",GLO,3.513587
115,acetaldehyde,1M-4,"United States, Gulf Coast",2021 Q3,1976.0,ACETALDEHYDE FROM ETHYLENE BY ONE-STEP OXIDATION,KG,136.077711,156.556828,Product,...,98.009256,6.066668,104.075924,17.257436,121.333361,146.166480,acetaldehyde,This dataset represents the production of 1 kg...,RER,1.498635
137,acetic anhydride,1M-9,"United States, Gulf Coast",2021 Q3,1973.0,ACETIC ANHYDRIDE FROM ACETALDEHYDE BY DIRECT O...,KG,226.796185,473.499448,Product,...,178.865768,11.062231,189.927999,31.316628,221.244627,145.505093,acetic anhydride,This data represents the production of 1 kg of...,RER,3.501515
141,acetic anhydride,1M-10,"United States, Gulf Coast",2021 Q3,1982.0,ACETIC ANHYDRIDE FROM ACETIC ACID VIA KETENE,KG,226.796185,231.718889,Product,...,183.274920,10.452658,193.727578,15.325581,209.053159,145.505093,acetic anhydride,This data represents the production of 1 kg of...,RER,3.501515
145,acetic anhydride,1M-11,"United States, Gulf Coast",2021 Q3,1973.0,ACETIC ANHYDRIDE FROM ACETONE VIA KETENE,KG,226.796185,261.805517,Product,...,215.053177,12.229929,227.283106,17.315471,244.598577,145.505093,acetic anhydride,This data represents the production of 1 kg of...,RER,3.501515
149,acetic anhydride,1M-812,"United States, Gulf Coast",2021 Q3,1994.0,"ACETIC ANHYDRIDE FROM MEOH, ACETIC ACID, AND C...",KG,174.633062,352.220917,Product,...,120.783542,7.949334,128.732876,30.253800,158.986676,145.505093,acetic anhydride,This data represents the production of 1 kg of...,RER,3.501515
154,isopropanol,1M-1351,"United States, Gulf Coast",2021 Q3,2011.0,ACETONE HYDROGENATION TO IPA IN PHENOL MANUFAC...,KG,117.934016,27.703104,Product,...,162.155710,8.719961,170.875672,3.523551,174.399223,111.444591,isopropanol,This dataset represents the production of 1 kg...,RoW,2.272976
156,acetonitrile,1M-14,"United States, Gulf Coast",2021 Q3,1991.0,ACETONITRILE RECOVERY IN ACRYLONITRILE MANUFAC...,KG,5.443108,42.856152,Product,...,174.702332,32.533821,207.236152,118.102054,325.338206,213.275192,acetonitrile,This dataset represents the multioutput-proces...,RER,4.199721
